In [89]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane_qiskit import IBMQDevice
from pennylane_qiskit import BasicAerDevice
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


import time
start = time.time()

In [90]:
# Read out CSV and sets/samples creation

df = pd.read_csv('fraud_detection_bank_dataset.csv', sep=',')
df = df.astype(float)
df = df.drop(['Unnamed: 0'], axis = 1)
df_sample = df.sample(2000)
train,test = train_test_split(df_sample, test_size=0.30, random_state=10)
train_set = train
test_set = test
np.random.seed(42)

In [91]:
# Review the information related to the dataframe

df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1989 to 15622
Columns: 113 entries, col_0 to targets
dtypes: float64(113)
memory usage: 1.7 MB


In [92]:
# Table of the description of the dataframe related to fixed parameters

df_sample.describe()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,targets
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.0,2000.0,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.0,2000.000000,2000.000000,2000.000000,2000.000000
mean,3.363500,319.388500,0.447000,2.431500,0.098500,1.058000,2.415000,3.363500,0.0,0.0,...,0.007000,0.354000,0.003500,0.328500,0.207000,0.0,0.054500,0.033000,45.240000,0.257500
std,13.707491,697.144656,5.222818,11.643285,1.393115,4.840697,2.983325,13.707491,0.0,0.0,...,0.089191,0.478329,0.059072,0.469785,0.405257,0.0,0.227058,0.354928,64.505317,0.437366
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,0.000000,39.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,5.000000,0.000000
50%,0.000000,105.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,20.000000,0.000000
75%,2.000000,302.000000,0.000000,2.000000,0.000000,1.000000,6.000000,2.000000,0.0,0.0,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,62.000000,1.000000
max,342.000000,12179.000000,194.000000,360.000000,54.000000,161.000000,8.000000,342.000000,0.0,0.0,...,2.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,11.000000,747.000000,1.000000


In [93]:
train_set.shape

(1400, 113)

In [94]:
test_set.shape

(600, 113)

In [95]:
train_set = train_set.astype(float)
test_set = test_set.astype(float)

In [96]:
x_train = train_set
y_train = train_set[['targets']]
x_test = test_set
y_test = test_set[['targets']]

x_train.drop(['targets'],axis = 1,inplace = True)
x_test.drop(['targets'],axis = 1,inplace = True)

In [97]:
#Train classes distribution
y_train.value_counts(normalize=True)*100

targets
0.0        74.142857
1.0        25.857143
dtype: float64

In [98]:
#Testing classes distribution
y_test.value_counts(normalize=True)*100

targets
0.0        74.5
1.0        25.5
dtype: float64

In [99]:
#Train and test shape
print("Train Shape: {}\nTest Shape : {}".format(x_train.shape, x_test.shape))

Train Shape: (1400, 112)
Test Shape : (600, 112)


In [100]:
#Dimension definition
n_dim = 2

#PCA application for dimensionality reduction
pca = PCA(n_components=n_dim, svd_solver='full')
pca.fit(x_train)
x_train_pca = pca.transform(x_train)
pca.fit(x_test)
x_test = pca.transform(x_test)

In [101]:
#Normalization of train
data = normalize(x_train_pca)
print(data)

[[-0.99997061  0.00766626]
 [-0.99995283 -0.00971271]
 [-0.99994675 -0.01032004]
 ...
 [-0.99744184  0.07148264]
 [-0.99997117 -0.00759275]
 [ 0.99997905 -0.0064727 ]]


In [102]:
#Normalization of test
x_test = normalize(x_test)
print(x_test)

[[ 0.999983   -0.00583141]
 [-0.99998912 -0.00466432]
 [ 0.99998742 -0.00501584]
 ...
 [-0.99999334 -0.00364874]
 [-0.99999382 -0.00351611]
 [-0.99999344 -0.00362247]]


In [103]:
# Angle Encoding

num_qubits = data.shape[1]

dev = qml.device('default.qubit', wires = num_qubits, shots = 256)
#dev = qml.device('qiskit.ibmq', wires = num_qubits, backend='ibmq_manila', provider=provider)

@qml.qnode(dev)
def circuit(parameters, data):
    # Apply Hadamards to all qubits in the circuit
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [104]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [105]:
circuit(weights_init, data[0])

tensor(-0.03125, requires_grad=True)

In [106]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [107]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [108]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [109]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [110]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.99997061, requires_grad=False), tensor(0.00766626, requires_grad=False)], Y = -1
X = [tensor(-0.99995283, requires_grad=False), tensor(-0.00971271, requires_grad=False)], Y = -1
X = [tensor(-0.99994675, requires_grad=False), tensor(-0.01032004, requires_grad=False)], Y = -1
X = [tensor(0.99467012, requires_grad=False), tensor(-0.10310845, requires_grad=False)], Y = -1
X = [tensor(-0.99995653, requires_grad=False), tensor(-0.00932417, requires_grad=False)], Y = -1


In [111]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
#opt = AdagradOptimizer(stepsize=0.01, eps=1e-08)
#opt = GradientDescentOptimizer(stepsize=0.01)
#opt = RMSPropOptimizer(stepsize=0.01, decay=0.9, eps=1e-08)
#opt = NesterovMomentumOptimizer(stepsize=0.01, momentum=0.9)
batch_size = 10

In [112]:
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(50):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 0.8142133 | Accuracy: 0.7371429 
New best
Iter:     2 | Cost: 0.7492751 | Accuracy: 0.7414286 
Iter:     3 | Cost: 0.7481851 | Accuracy: 0.7414286 
Iter:     4 | Cost: 0.8636719 | Accuracy: 0.6300000 
Iter:     5 | Cost: 0.8025046 | Accuracy: 0.7292857 
Iter:     6 | Cost: 0.7605707 | Accuracy: 0.7414286 
Iter:     7 | Cost: 0.7525719 | Accuracy: 0.7414286 
Iter:     8 | Cost: 0.7460470 | Accuracy: 0.7414286 
Iter:     9 | Cost: 0.7504315 | Accuracy: 0.7414286 
Iter:    10 | Cost: 0.7668446 | Accuracy: 0.7414286 
Iter:    11 | Cost: 0.7903816 | Accuracy: 0.7414286 
Iter:    12 | Cost: 0.7583754 | Accuracy: 0.7414286 
Iter:    13 | Cost: 0.7435417 | Accuracy: 0.7414286 
Iter:    14 | Cost: 0.8243844 | Accuracy: 0.7314286 
Iter:    15 | Cost: 0.9313670 | Accuracy: 0.4850000 
Iter:    16 | Cost: 0.9178925 | Accuracy: 0.4914286 
Iter:    17 | Cost: 0.8572023 | Accuracy: 0.7028571 
Iter:    18 | Cost: 0.7702813 | Accuracy: 0.7414286 
Iter:    19 | Cost: 0.774080

In [113]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [114]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.7525858753099877, Accuracy: 74.0%


In [115]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,-1.0,-1.0
1,-1.0,-1.0
2,-1.0,1.0
3,-1.0,-1.0
4,-1.0,-1.0
...,...,...
595,-1.0,1.0
596,-1.0,-1.0
597,-1.0,-1.0
598,-1.0,1.0


In [116]:
end = time.time()
totaltime = end - start

mins = int(np.round(totaltime % 60))
secs = int(np.round((totaltime % 60 - mins) * 60))

print(f'Execution time: {mins}m{secs}s')

Execution time: 32m-11s


In [117]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))
print(metrics.confusion_matrix(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       1.00      0.74      0.85       600
         1.0       0.00      0.00      0.00         0

    accuracy                           0.74       600
   macro avg       0.50      0.37      0.43       600
weighted avg       1.00      0.74      0.85       600

0.0
0.0
0.0
0.745
[[447 153]
 [  0   0]]


c:\Users\USER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\USER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\USER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\USER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: